In [1]:
#CSV Bibliothek importieren 
import csv
#Pandas importieren und als pd bezeichnen
import pandas as pd
#Numpy importieren und als np bezeichnen 
import numpy as np
#Json Biliothek importieren
import json
#Obere Schranke für die Größe der Daten auf 100,000,000 setzen  
csv.field_size_limit(100000000)
#Maximale Anzahl von Zeilen und Spalten Zeigen 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#Extraktion der Domäne
import tldextract
#Um die Warnings zu Console leiten
import warnings
warnings.filterwarnings('ignore')

***

# Politische Polarisierung

1. Die Liste der polarisierten Domäne werden vorbereitet.
2. Die Anzahl der benutzten polarisierten Domäne in den Tweets wird berechnet
3. Die Polarisierung für die Benutzer wird berechnet
4. Zum Schluss werden die Ergebnisse gespeichert.

***

### Zuerst werden die Tweets in TweetsCov19 bearbeitet, dann die Tweets in TweetsKB
#### Vorbereitung der Domain-Labels

In [2]:
#Hier werden zuerst die Domain-Labels gelesen
pfad_tsv2 = 'C:/Mo/tweetDB/data/labels.csv'

df_label = pd.read_csv(pfad_tsv2)


In [3]:
#Die Quellen und Kategorien, die durch die Quelle unterstützt werden.
pd.options.display.max_columns = None
df_label.head()

,Unnamed: 0,"NewsGuard, Does not repeatedly publish false content","NewsGuard, Gathers and presents information responsibly","NewsGuard, Regularly corrects or clarifies errors","NewsGuard, Handles the difference between news and opinion responsibly","NewsGuard, Avoids deceptive headlines","NewsGuard, Website discloses ownership and financing","NewsGuard, Clearly labels advertising","NewsGuard, Reveals who's in charge, including any possible conflicts of interest","NewsGuard, Provides information about content creators","NewsGuard, score","NewsGuard, overall_class","Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake","Open Sources, reliable","Open Sources, fake","Open Sources, unreliable","Open Sources, bias","Open Sources, conspiracy","Open Sources, hate","Open Sources, junksci","Open Sources, rumor","Open Sources, blog","Open Sources, clickbait","Open Sources, political","Open Sources, satire","Open Sources, state","Media Bias / Fact Check, label","Media Bias / Fact Check, factual_reporting","Media Bias / Fact Check, extreme_left","Media Bias / Fact Check, right","Media Bias / Fact Check, extreme_right","Media Bias / Fact Check, propaganda","Media Bias / Fact Check, fake_news","Media Bias / Fact Check, some_fake_news","Media Bias / Fact Check, failed_fact_checks","Media Bias / Fact Check, conspiracy","Media Bias / Fact Check, pseudoscience","Media Bias / Fact Check, hate_group","Media Bias / Fact Check, anti_islam","Media Bias / Fact Check, nationalism","Allsides, bias_rating","Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stCenturyWire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
1,ABC News,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_center_bias,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lean Left,8964.0,6949.0,somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMERICAblog News,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
3,Activist Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
4,Addicting Info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,left_bias,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Die Spalte 0 wird zu Domain umbenannt.
df_label.rename(columns = {'Unnamed: 0' : 'domain'}, inplace = True)
df_label['domain'] = df_label['domain'].str.replace(" ", "")
df_label['domain'] = df_label['domain'].str.lower()

In [5]:
df_label.head(5)

,domain,"NewsGuard, Does not repeatedly publish false content","NewsGuard, Gathers and presents information responsibly","NewsGuard, Regularly corrects or clarifies errors","NewsGuard, Handles the difference between news and opinion responsibly","NewsGuard, Avoids deceptive headlines","NewsGuard, Website discloses ownership and financing","NewsGuard, Clearly labels advertising","NewsGuard, Reveals who's in charge, including any possible conflicts of interest","NewsGuard, Provides information about content creators","NewsGuard, score","NewsGuard, overall_class","Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake","Open Sources, reliable","Open Sources, fake","Open Sources, unreliable","Open Sources, bias","Open Sources, conspiracy","Open Sources, hate","Open Sources, junksci","Open Sources, rumor","Open Sources, blog","Open Sources, clickbait","Open Sources, political","Open Sources, satire","Open Sources, state","Media Bias / Fact Check, label","Media Bias / Fact Check, factual_reporting","Media Bias / Fact Check, extreme_left","Media Bias / Fact Check, right","Media Bias / Fact Check, extreme_right","Media Bias / Fact Check, propaganda","Media Bias / Fact Check, fake_news","Media Bias / Fact Check, some_fake_news","Media Bias / Fact Check, failed_fact_checks","Media Bias / Fact Check, conspiracy","Media Bias / Fact Check, pseudoscience","Media Bias / Fact Check, hate_group","Media Bias / Fact Check, anti_islam","Media Bias / Fact Check, nationalism","Allsides, bias_rating","Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stcenturywire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
1,abcnews,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_center_bias,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lean Left,8964.0,6949.0,somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,americablognews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
3,activistpost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
4,addictinginfo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,left_bias,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Die beinhalteten Values für Media Bias,Fact Check zählen 
df_label['Media Bias / Fact Check, label'].value_counts()

left_center_bias            36
left_bias                   30
conspiracy_pseudoscience    17
right_bias                  16
questionable_source         15
right_center_bias           10
satire                       8
least_biased                 3
Name: Media Bias / Fact Check, label, dtype: int64

In [9]:
#Die beinhalteten Values für BuzzFeed, leaning zählen 
df_label['BuzzFeed, leaning'].value_counts()

right    32
left     24
Name: BuzzFeed, leaning, dtype: int64

In [10]:
#Die beinhalteten Values für Allsides, bias_rating zählen 
df_label['Allsides, bias_rating'].value_counts()

Left          17
Right         14
Lean Left     13
Center        11
Lean Right    10
Name: Allsides, bias_rating, dtype: int64

In [11]:
#Die beinhalteten Values für Pew Research Center, consistently_liberal zählen 
df_label['Pew Research Center, consistently_liberal'].value_counts()

 1.0    20
-1.0     5
Name: Pew Research Center, consistently_liberal, dtype: int64

In [12]:
#Die beinhalteten Values für Pew Research Center, mostly_liberal zählen 
df_label['Pew Research Center, mostly_liberal'].value_counts()

 1.0    17
-1.0     5
 0.0     3
Name: Pew Research Center, mostly_liberal, dtype: int64

In [13]:
#Die beinhalteten Values für Pew Research Center, mostly conservative zählen 
df_label['Pew Research Center, mostly conservative'].value_counts()

-1.0    12
 1.0     7
 0.0     6
Name: Pew Research Center, mostly conservative, dtype: int64

In [14]:
#Die beinhalteten Values für Pew Research Center, consistently conservative zählen 
df_label['Pew Research Center, consistently conservative'].value_counts()

-1.0    19
 1.0     4
 0.0     2
Name: Pew Research Center, consistently conservative, dtype: int64

In [15]:
#Die beinhalteten Values für Pew Research Center, mixed zählen 
df_label['Pew Research Center, mixed'].value_counts()

 1.0    12
 0.0    10
-1.0     3
Name: Pew Research Center, mixed, dtype: int64

In [16]:
#Die beinhalteten Values für Pew Research Center, mixed zählen 
df_label['Media Bias / Fact Check, extreme_left'].value_counts()

0.0    133
1.0      2
Name: Media Bias / Fact Check, extreme_left, dtype: int64

In [17]:
#Die beinhalteten Values für Media Bias / Fact Check, extreme_right zählen 
df_label['Media Bias / Fact Check, extreme_right'].value_counts()

0.0    123
1.0     12
Name: Media Bias / Fact Check, extreme_right, dtype: int64

In [18]:
#Die beinhalteten Values für Media Bias / Fact Check, right zählen 
df_label['Media Bias / Fact Check, right'].value_counts()

0.0    134
1.0      1
Name: Media Bias / Fact Check, right, dtype: int64

- Liberale Domäne:

In [20]:
#Die Labels von Media Bias / Fact Check, label, die mit left_bias markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, label'] == 'left_bias']

In [21]:
#die Labels zu Liste umwandeln und speichern.
list_lib = []
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_lib.extend(l_Media_Bias)

In [22]:
#Die Labels von Media Bias / Fact Check, label, die mit left_center_bias markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, label'] == 'left_center_bias']

In [23]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_lib.extend(l_Media_Bias)

In [24]:
#Die Labels von BuzzFeed, leaning, die mit left markiert sind, auswählen.
l_Media_Bias = df_label[df_label['BuzzFeed, leaning'] == 'left']

In [25]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_lib.extend(l_Media_Bias)

In [26]:
#Die Labels von Allsides, bias_rating, die mit left markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Allsides, bias_rating'] == 'Left']

In [27]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_lib.extend(l_Media_Bias)

In [28]:
#Die Labels von Allsides, bias_rating, die mit Lean left markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Allsides, bias_rating'] == 'Lean Left']

In [29]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_lib.extend(l_Media_Bias)

In [30]:
#Die Labels von Pew Research Center, consistently_liberal, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Pew Research Center, consistently_liberal'] == 1.0]

In [31]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_lib.extend(l_Media_Bias)

In [32]:
#Die Labels von Pew Research Center, mostly_liberal, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Pew Research Center, mostly_liberal'] == 1.0]

In [33]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_lib.extend(l_Media_Bias)

In [34]:
#Die Labels von Pew Research Center, mixed, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Pew Research Center, mixed'] == 1.0]

In [35]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_lib.extend(l_Media_Bias)

In [38]:
#Die Labels von Media Bias / Fact Check, extreme_left, die mit extreme_left markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, extreme_left'] == 1.0]

In [39]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_lib.extend(l_Media_Bias)

In [8]:
list_lib.append('')

In [9]:
list_lib = np.array(list_lib)
list_lib = np.unique(list_lib)
list_lib = list_lib.tolist()

In [10]:
#Anzahl der liberalen Domäne
len(list_lib)

170

In [19]:
#Liste der Liberalen
list_lib = np.array(list_lib)
list_lib.reshape(1,-1)

array([['21stcenturywire', '604now', '972mag', 'abc11', 'abcnews',
        'aceshowbiz', 'achnews', 'aclu', 'act', 'activistpost',
        'addictinginfo', 'aeon', 'affinitymagazine', 'afj', 'africanews',
        'aidc', 'aljazeera', 'allgov', 'alternet', 'americablog',
        'americablognews', 'americanbridgepac', 'americannewsx',
        'americanprogress', 'angrywhitemen', 'antifascistnews',
        'attackthesystem', 'au', 'autostraddle', 'bbc', 'billmoyers',
        'bipartisanreport', 'birminghammail', 'bitchmedia',
        'blackagendareport', 'blackamericaweb', 'blacklivesmatter',
        'blackmainstreet', 'bluestatedaily', 'boingboing',
        'bostonreview', 'businessinsider', 'bust', 'buzzfeed', 'c4ss',
        'calltoactivism', 'canadafactcheck', 'canadiandimension',
        'capitalandmain', 'care2', 'cbsnews', 'ceasefiremagazine',
        'change', 'chicagoreader', 'chicagoreporter', 'chicagosun-times',
        'citizencritics', 'citypaper', 'cnbc', 'cnn', 'codepink',

In [12]:
#Liste speichern.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_liberal_2.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for i  in range (0, len(list_lib)):
        l = []
        l.append(list_lib[i])
        tsv_writer.writerow(l)

- Konservative Domäne:

In [94]:
#Die Labels von Media Bias / Fact Check, label, die mit right_center_bias markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, label'] == 'right_center_bias']

In [95]:
#die Labels zu Liste umwandeln und speichern.
list_con = []
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [89]:
#Die Labels von Media Bias / Fact Check, label, die mit right_bias markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, label'] == 'right_bias']

In [90]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [91]:
#Die Labels von Media Bias / Fact Check, label, die mit least_biased markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, label'] == 'least_biased']

In [92]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [93]:
#Die Labels von BuzzFeed, leaning, die mit right markiert sind, auswählen.
l_Media_Bias = df_label[df_label['BuzzFeed, leaning'] == 'right']

In [94]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [95]:
#Die Labels von Allsides, bias_rating, die mit right markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Allsides, bias_rating'] == 'Right']

In [96]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [97]:
#Die Labels von Allsides, bias_rating, die mit Lean right markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Allsides, bias_rating'] == 'Lean Right']

In [98]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [99]:
#Die Labels von Allsides, bias_rating, die mit Lean Center markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Allsides, bias_rating'] == 'Center']

In [100]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [101]:
#Die Labels von Pew Research Center, mostly conservative, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Pew Research Center, mostly conservative'] == 1.0]

In [102]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [103]:
#Die Labels von Pew Research Center, mostly conservative, die mit 0.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Pew Research Center, mostly conservative'] == 0.0]

In [104]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [105]:
#Die Labels von Pew Research Center, mostly conservative, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Pew Research Center, consistently conservative'] == 1.0]

In [106]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [107]:
#Die Labels von Pew Research Center, consistently conservative, die mit 0.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Pew Research Center, consistently conservative'] == 0.0]

In [108]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [109]:
#Die Labels von Pew Research Center, mixed, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Pew Research Center, mixed'] == 1.0]

In [110]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [117]:
#Die Labels von Media Bias / Fact Check, extreme_right, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, extreme_right'] == 1.0]

In [118]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [119]:
#Die Labels von Media Bias / Fact Check, right, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, right'] == 1.0]

In [120]:
#die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_con.extend(l_Media_Bias)

In [59]:
list_con.append('hedailyliberator')

In [60]:
list_con = np.array(list_con)
list_con = np.unique(list_con)
list_con = list_con.tolist()

In [61]:
#Anzahl der Konservative Domäne
len(list_con)

170

In [62]:
#Liste der konservativen Domäne
list_con = np.array(list_con)
list_con.reshape(1,-1)

array([['2ndvote', 'Aish', 'AsianNews', 'CAGW', 'aa', 'abcnews',
        'ac2news', 'academia', 'aclj', 'aei', 'afpc', 'aim', 'aina',
        'alec', 'aljazeera', 'altnewsmedia', 'amac',
        'americanconsequences', 'americanexperiment',
        'americanlibertyreport', 'americansforprosperity',
        'americarisingpac', 'amgreatness', 'amimagazine', 'ammoland',
        'arizonadailyindependent', 'armstrongeconomics',
        'axiomnewsnetwork', 'bearingarms', 'beliefnet', 'bizpacreview',
        'blabber', 'boundingintocomics', 'breitbart', 'brexitcentral',
        'businessinsider', 'buzzfeed', 'buzzsawpolitics', 'c2cjournal',
        'ca-political', 'calgarysun', 'campaignlifecoalition',
        'campusreform', 'capitalresearch', 'cbsnews', 'cc',
        'chicksontheright', 'christianheadlines', 'christiannewsalerts',
        'christianpost', 'christiantoday', 'chroniclesmagazine',
        'citizenfreepress', 'citizensunited', 'city-journal',
        'claremont', 'cnbc', 'cnsne

In [63]:
#Liste der Konservativen speichern.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_conservative_2.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for i  in range (0, len(list_con)):
        l = []
        l.append(list_con[i])
        tsv_writer.writerow(l)

***

## Polarisierungen berechnen
### TweetsCov19

***

In [2]:
#Die liste der liberalen Domäne lesen.
pfad = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_liberal_2.tsv'

df_dom_lib = pd.read_csv(pfad, sep='\t', error_bad_lines=False, names=['dom'])

In [3]:
#Die Spalte der anti-wissenschaftliche Domöne zu Liste umwandeln.
list_lib = df_dom_lib.dom.tolist()

In [4]:
list_lib = np.array(list_lib)
list_lib = np.unique(list_lib)
list_lib = list_lib.tolist()

In [5]:
len(list_lib)

170

In [26]:
#Die liste der konservativen Domäne lesen.
pfad = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_conservative_2.tsv'

df_dom_con = pd.read_csv(pfad, sep='\t', error_bad_lines=False, names=['dom'])

In [27]:
#Die Spalte der pro-wissenschaftliche Domäne zu Liste umwandeln.
list_con = df_dom_con.dom.tolist()

In [28]:
list_con = np.array(list_con)
list_con = np.unique(list_con)
list_con = list_con.tolist()

In [29]:
len(list_con)

170

In [21]:
#Alle tweets als Dataframe lesen.
pfad_tsv1 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/url/tweetsCov19_id_is_dom.tsv'

df_tweetscov19 = pd.read_csv(pfad_tsv1, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Username','is_deleted','Urls_num', 'Url_dom'])
pd.options.display.max_columns = None
print('Shape :', df_tweetscov19.shape)
df_tweetscov19.head()

Shape : (9409841, 5)


,Tweet_Id,Username,is_deleted,Urls_num,Url_dom
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,1,catholicnewsagency.com
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0


In [22]:
#Diese Methode bekommt benutzten Domäne als ein String.
#Wandelt sie die String-daten zu Liste um.
#Die Methode geht die Liste durch, prüft jede Domain. 
#Wenn Domain in liberalen Domäne wäre, dann bekommt der Tweet eine Wertung von -1.

def get_politik_pol(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        d = tldextract.extract(domains_list[i])
        d = d.domain
                   
        
        if d in list_lib:
            score -=  1
        
    return score

In [23]:
#Für alle Tweets wird die Anzahl der benutzten liberalen Domäne berechnet.
df_tweetscov19['Url_dom'] = df_tweetscov19['Url_dom'].astype(str)
df_tweetscov19['dom_neg_pol'] = df_tweetscov19.apply(lambda row: get_politik_pol(row['Url_dom']) if (row['Url_dom'] != '0') else 0, axis=1)

In [24]:
df_tweetscov19.head()

,Tweet_Id,Username,is_deleted,Urls_num,Url_dom,dom_neg_pol
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,1,catholicnewsagency.com,0
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0,0


In [25]:
#Anzahl der Tweets die eine Wertung weniger als 0 haben.
df_tweetscov19[df_tweetscov19['dom_neg_pol'] < 0].shape

(264919, 6)

In [26]:
#Diese Methode bekommt benutzten Domäne als ein String.
#Wandelt sie die String-daten zu Liste um.
#Die Methode geht die Liste durch, prüft jede Domain. 
#Wenn Domain in konservative Domäne wäre, dann bekommt der Tweet eine Wertung von 1.

def get_politik_pol_pos(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        d = tldextract.extract(domains_list[i])
        d = d.domain
       
            
        
        if d in list_con:
            score +=  1
        
    return score

In [27]:
#Für alle Tweets wird die Anzahl der benutzten konservativen Domäne berechnet.
df_tweetscov19['dom_pos_pol'] = df_tweetscov19.apply(lambda row: get_politik_pol_pos(row['Url_dom']) if (row['Url_dom'] != '0') else 0, axis=1)

In [28]:
df_tweetscov19.head()

,Tweet_Id,Username,is_deleted,Urls_num,Url_dom,dom_neg_pol,dom_pos_pol
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,1,catholicnewsagency.com,0,0
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0,0,0


In [29]:
df_tweetscov19[df_tweetscov19['dom_pos_pol'] > 0].shape

(270001, 7)

In [30]:
#Hier werden die Benutzer in Dataframe gruppiert, und die Features 'Anzahl der Domäne', 'Negative Wertungen' und 'Psoitive Wertungen summiert.' 
#dom_neg_pol = Liberalen
#dom_pos_pol = Konservativen
df_tweetscov19_unique_user = df_tweetscov19.groupby('Username')['Urls_num', 'dom_neg_pol', 'dom_pos_pol'].sum()

In [31]:
#Anzahl der einzigartigen Users in Dataframe
df_tweetscov19_unique_user.shape

(4095876, 3)

In [32]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['dom_pos_pol']>0].head()

,Urls_num,dom_neg_pol,dom_pos_pol
Username,,,
00012a177385cd6e7590e2acb3f206e3,1,-1,1
00016e20a3865cb74581803c27d27b89,1,-1,1
000220d9d43b6b1ad7dbbdbfbcc53371,5,-1,2
00037260d5e4fe01965a48a96974b53f,2,-2,2
0003fb847751a16fae91117947a259b7,2,-1,1


In [33]:
#Anzahl der benutzten polarisierten Domäne wird berechnet.
df_tweetscov19_unique_user['pol_num'] = df_tweetscov19_unique_user.apply(lambda row: (-1 * (row['dom_pos_pol'] - row['dom_neg_pol'])) if (row['dom_pos_pol'] - row['dom_neg_pol']) < 0 else (row['dom_pos_pol'] - row['dom_neg_pol']), axis=1)

In [34]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['dom_neg_pol']<0].head()

,Urls_num,dom_neg_pol,dom_pos_pol,pol_num
Username,,,,
00008ca06f863b962f72cc9167d7978a,1,-1,0,1
00012a177385cd6e7590e2acb3f206e3,1,-1,1,2
00016e20a3865cb74581803c27d27b89,1,-1,1,2
0001a863c159122c712c53258cf13c8b,1,-1,0,1
000220d9d43b6b1ad7dbbdbfbcc53371,5,-1,2,3


In [35]:
#Die Polarisierung der Domäne wird berechnet.
#Anzahl der Negativen + der Positiven durch die Anzahl der benutzten Domäne.
df_tweetscov19_unique_user['user_pol'] = df_tweetscov19_unique_user.apply(lambda row: (row['dom_neg_pol'] + row['dom_pos_pol']) / row['pol_num'] if row['pol_num'] != 0 else 0, axis=1)

In [36]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['dom_neg_pol']<-2].head()

,Urls_num,dom_neg_pol,dom_pos_pol,pol_num,user_pol
Username,,,,,
00065a00b971b5e1629d504e0ddf2da0,27,-5,9,14,0.285714
0013aaa4da0461c58d0b1306af75af02,215,-16,35,51,0.372549
00247be59d748ab0e377ab5885ecc66b,7,-4,2,6,-0.333333
0028b1c0f33fc484e49c2ce2820cd4ae,25,-4,4,8,0.000000
002f506ad2fa8c2cb570fa65d135de67,24,-20,0,20,-1.000000


In [37]:
#Die Benutzer, die eine Wertung von 0 bekommen haben, werden ausgenommen.
df_tweetscov19_unique_user_0 = df_tweetscov19_unique_user[df_tweetscov19_unique_user['user_pol'] != 0]

In [38]:
#Die Form der Dataframe
df_tweetscov19_unique_user_0.shape

(52631, 5)

In [39]:
#Die Form der Dataframe Konservativen.
df_tweetscov19_unique_user_0[df_tweetscov19_unique_user_0['user_pol'] > 0].shape

(21545, 5)

In [40]:
#Die Form der Dataframe Liberalen.
df_tweetscov19_unique_user_0[df_tweetscov19_unique_user_0['user_pol'] < 0].shape

(31086, 5)

In [381]:
#Die polarisieten Werte für Tweets werden gespeichert.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/tweetscov19_dom_politik.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19.iterrows():
        tsv_writer.writerow(row.values.tolist())

In [ ]:
df_tweetscov19_unique_user_0 = df_tweetscov19_unique_user_0.reset_index()

In [41]:
#Die Benutzer mit bekannten polarisierten Ansichten speichern. 
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/users/tweetscov19_dom_politik.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19_unique_user_0.iterrows():
        tsv_writer.writerow(row.values.tolist())

****

## Polarisierungen berechnen
### TweetsKB

***

In [42]:
#Dataframe wird gelesen.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/tweetskb_sample/url/tweetskb_sample_id_is_urls.tsv'

df_tweetskb = pd.read_csv(pfad_tsv3, sep='\t', error_bad_lines=False, names=['Tweet_Id', "Username", 'is_deleted', 'URLs_dom', 'URLs_num'])
pd.options.display.max_columns = None
print('Shape :', df_tweetskb.shape)
df_tweetskb.head()

Shape : (9409841, 5)


,Tweet_Id,Username,is_deleted,URLs_dom,URLs_num
0,1178791636039589889,DCComics,False,dcuniverseinfinite.com,1
1,1178791641265639424,seanberdyz,True,carrd.co,1
2,1178791640573579264,AnArtistAtBirth,True,0,0
3,1178791644797255680,lagosboygang,True,0,0
4,1178791648056217600,Phenom_Hoops,False,phenomhoopreport.com,1


In [43]:
#Für alle Domäne in Tweets wird die Anzahl der liberalen Domäne berechnet.
df_tweetskb['URLs_dom'] = df_tweetskb['URLs_dom'].astype(str)
df_tweetskb['dom_neg_pol'] = df_tweetskb.apply(lambda row: get_politik_pol(row['URLs_dom']) if (row['URLs_dom'] != '0') else 0, axis=1)

In [44]:
df_tweetskb[df_tweetskb['dom_neg_pol'] < 0].shape

(56960, 6)

In [45]:
#Für alle Domäne in Tweets wird die Anzahl der konservativen Domäne berechnet.
df_tweetskb['dom_pos_pol'] = df_tweetskb.apply(lambda row: get_politik_pol_pos(row['URLs_dom']) if (row['URLs_dom'] != '0') else 0, axis=1)

In [46]:
df_tweetskb[df_tweetskb['dom_pos_pol'] > 0].shape

(58511, 7)

In [47]:
#Hier werden die Benutzer in Dataframe gruppiert, und die Features 'Anzahl der Domäne', 'Negative Wertungen' und 'Psoitive Wertungen summiert.' 
#Negativ = Liberalen
#Positiv = Konservativen
df_tweetskb_unique_user = df_tweetskb.groupby('Username')['URLs_num', 'dom_neg_pol', 'dom_pos_pol'].sum()

In [48]:
df_tweetskb_unique_user.shape

(5208425, 3)

In [49]:
df_tweetskb_unique_user.head()

,URLs_num,dom_neg_pol,dom_pos_pol
Username,,,
00000000,0,0,0
000000000yt,0,0,0
00000000martin,0,0,0
000000911a,0,0,0
000000_127,0,0,0


In [50]:
#Die Anzahl der benutzten polarisierten Domäne wird berechnet.
df_tweetskb_unique_user['pol_num'] = df_tweetskb_unique_user.apply(lambda row: (-1 * (row['dom_pos_pol'] - row['dom_neg_pol'])) if (row['dom_pos_pol'] - row['dom_neg_pol']) < 0 else (row['dom_pos_pol'] - row['dom_neg_pol']), axis=1)

In [51]:
df_tweetskb_unique_user[df_tweetskb_unique_user['dom_neg_pol']<-2].head()

,URLs_num,dom_neg_pol,dom_pos_pol,pol_num
Username,,,,
0nehunter69,4,-3,0,3
1120Eclipse,20,-10,9,19
11thHour,14,-9,0,9
13SciDave,4,-3,3,6
1944jnewell,3,-3,3,6


In [52]:
#Die Polarisierungswerte für die Benutzer wird berechnet.
df_tweetskb_unique_user['user_pol'] = df_tweetskb_unique_user.apply(lambda row: (row['dom_neg_pol'] + row['dom_pos_pol']) / row['pol_num'] if row['pol_num'] != 0 else 0, axis=1)

In [53]:
#Die Benutzer, die eine Wertung von 0 bekommen haben, werden ausgenommen.
df_tweetskb_unique_user_0 = df_tweetskb_unique_user[df_tweetskb_unique_user['user_pol'] != 0]

In [54]:
#Die Form der Dataframe
df_tweetskb_unique_user_0.shape

(17303, 5)

In [55]:
#Die liberalen Benutzer 
df_tweetskb_unique_user_0[df_tweetskb_unique_user_0['user_pol'] > 0].shape

(7984, 5)

In [56]:
#Die konservativen Benutzer 
df_tweetskb_unique_user_0[df_tweetskb_unique_user_0['user_pol'] < 0].shape

(9319, 5)

In [382]:
#Die polarisierten Werte für die Tweets werden gespeichert.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/tweetskb_dom_politik.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb.iterrows():
        tsv_writer.writerow(row.values.tolist())

In [ ]:
df_tweetskb_unique_user_0 = df_tweetskb_unique_user_0.reset_index()

In [57]:
#Die Benutzer mit polarisierten Werte werden gespeichert.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/users/tweetskb_dom_politik.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb_unique_user_0.iterrows():
        tsv_writer.writerow(row.values.tolist())